In [7]:
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

currentPath = os.getcwd()
dataPath = (currentPath + '/../data/external')
# Read the two CRSP csv files. 
crspmret = pd.read_csv(dataPath + '/crspmret.csv')
crspminfo = pd.read_csv(dataPath + '/crspminfo.csv')


In [8]:
a = 5

In [11]:
try:
    x = 1
    y = 0
    assert y != 0, "Invalid Operation"
    print(x / y)
 
# the errror_message provided by the user gets printed
except AssertionError as msg:
    print(msg)

print('Ciao')

Invalid Operation
Ciao


Momentum for stock *i* at month *t* is given by the cumulated return from t-11 to t-1.
We require at least 9 months of observations in the previous 12 months otherwise the entry is considered missing.

In [3]:
# Read the signals dataframes
df = pd.read_csv(dataPath+'/signed_predictors_all_wide.csv', usecols=['permno','yyyymm','Mom12m'])
crsp = crspminfo.merge(crspmret, on=['permno','yyyymm']).drop('date',axis=1)
crsp = crsp.merge(df, on=['permno', 'yyyymm'])#.dropna(subset=['BMdec'])


In [ ]:
crsp.dropna(subset=['Mom12m'], inplace=True)
del df

In [9]:
crsp

,permno,yyyymm,prc,exchcd,me,shrcd,siccd,me_nyse10,me_nyse20,ret,melag,Mom12m
10,10000,198612,-0.515625,3.0,1981.546875,10.0,3990.0,57007.3875,222725.025,-37.735850,3182.484375,-0.810714
11,10000,198701,-0.406250,3.0,1581.531250,10.0,3990.0,61945.8375,244496.175,-21.212122,1981.546875,-0.882143
12,10000,198702,-0.406250,3.0,1581.531250,10.0,3990.0,71816.4750,261162.050,0.000000,1581.531250,-0.875000
13,10000,198703,-0.250000,3.0,973.250000,10.0,3990.0,65248.5750,256194.400,-38.461539,1581.531250,-0.908451
14,10000,198704,-0.234375,3.0,912.421875,10.0,3990.0,69094.4625,253489.625,-6.250000,973.250000,-0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...
48325,10281,199101,4.312500,3.0,29799.375000,11.0,7370.0,176016.5875,322074.300,-1.428571,30231.250000,0.076923
48326,10281,199102,3.937500,3.0,27499.500000,11.0,7370.0,148980.6125,317866.600,-8.695652,29799.375000,-0.158537
48327,10281,199103,4.187500,3.0,29245.500000,11.0,7370.0,178375.7125,349937.700,6.349207,27499.500000,-0.222222
48328,10281,199104,4.625000,3.0,32301.000000,11.0,7370.0,209634.5500,384802.950,10.447761,29245.500000,-0.162500


In [17]:
crsp['mom_decile'] = crsp.groupby('yyyymm')['Mom12m'].apply(lambda x: pd.qcut(x, 10, labels=False))


In [21]:
Mom = crsp[['permno','yyyymm','ret','melag','Mom12m','mom_decile']]

In [28]:
Mom['ret'] = Mom['ret']/100

/tmp/ipykernel_2115/3472354924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mom['ret'] = Mom['ret']/100


In [29]:
Mom

,permno,yyyymm,ret,melag,Mom12m,mom_decile
10,10000,198612,-0.377358,3182.484375,-0.810714,0
11,10000,198701,-0.212121,1981.546875,-0.882143,0
12,10000,198702,0.000000,1581.531250,-0.875000,0
13,10000,198703,-0.384615,1581.531250,-0.908451,0
14,10000,198704,-0.062500,973.250000,-0.937500,0
...,...,...,...,...,...,...
48325,10281,199101,-0.014286,30231.250000,0.076923,8
48326,10281,199102,-0.086957,29799.375000,-0.158537,5
48327,10281,199103,0.063492,27499.500000,-0.222222,4
48328,10281,199104,0.104478,29245.500000,-0.162500,4


In [34]:
test = Mom.groupby(['yyyymm','mom_decile']).apply(lambda x: np.sum(x['ret']*(x['melag']/(x['melag'].sum())))).reset_index()

In [41]:
test.pivot('yyyymm', 'mom_decile', values=0).iloc[:,:].mean()

mom_decile
0    0.006604
1    0.013696
2    0.007996
3    0.008857
4    0.012044
5    0.013408
6    0.011441
7    0.006990
8    0.015444
9    0.018628
dtype: float64

In [33]:
crsp.loc[crsp['yyyymm'] == 193601]

,permno,yyyymm,prc,exchcd,me,shrcd,siccd,me_nyse10,me_nyse20,ret,melag,Mom12m,mom_decile
1024,10006,193601,33.750,1.0,20250.000,10.0,3740.0,6352.5,7375.0,-0.369004,20325.000,0.935714,7
2453,10014,193601,16.375,1.0,24562.500,10.0,1000.0,6352.5,7375.0,8.455285,23062.500,-0.062903,0
3836,10022,193601,-94.000,1.0,16450.000,10.0,3420.0,6352.5,7375.0,3.296703,15925.000,0.386895,3
5664,10030,193601,47.500,1.0,29070.000,10.0,3310.0,6352.5,7375.0,11.764706,26010.000,0.668125,4
9824,10057,193601,14.750,1.0,7375.000,11.0,3540.0,6352.5,7375.0,0.854701,7312.500,1.489362,8
13054,10073,193601,3.875,1.0,790.500,10.0,3520.0,6352.5,7375.0,14.779875,4054.500,2.697675,9
16698,10102,193601,189.000,1.0,158949.000,10.0,2810.0,6352.5,7375.0,11.834320,142129.000,0.582170,4
21427,10137,193601,23.125,1.0,40491.875,11.0,4910.0,6352.5,7375.0,4.519774,38740.875,0.732519,5
23650,10145,193601,165.000,1.0,365310.000,11.0,2810.0,6352.5,7375.0,5.630452,348981.750,0.192828,1
25859,10153,193601,41.000,1.0,55104.000,10.0,3520.0,6352.5,7375.0,11.186441,49560.000,1.302265,7
